In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Data Analysis for "The Machine Learning Process Course on 365 Data Science". 
We are analyzing real data from my YouTube channel. The focus of EDA is to understand patterns in the data so that we can start generating insights and predictions. In this notebook, I walk through the main EDA concepts from the machine learning process course. We cover:
- Single Variable Plots
 - Histograms 
 - Box Plots
 - Bar charts
- Relationships & Multi-variable plots 
 - Scatterplots
 - Correlation Matrices 
 - Pivot Tables 
 - Bar Charts 
 - Line Charts
 
 
 I will be using matplotlib and Seaborn to visualize this data. It should be noted that there are plenty of different visualization libraries to choose from. I personally use plotly quite a lot in my own personal projects. I find that these two that we are using have good basics that are easy to understand and build on. 

## Loading in the data
Our first step is to load in the data and get a feel for what we will be working with. We do the following steps before we start with the true EDA:
1) Load in our libraries that we plan to use for data manipulation and visualization 

2) Load in our data 

3) Explore the high level features of our data (size, columns, etc.)

4) Additional cleaning of our data if needed 

5) Explore high level descriptive statistics of our data

In [2]:
#import basic visualization libraries 
import matplotlib.pyplot as plt 
import seaborn as sns  
#clean columns 

In [3]:
#read in data 
df_agg = pd.read_csv('/kaggle/input/ken-jee-youtube-data/Aggregated_Metrics_By_Video.csv',encoding='utf-8')
df_agg_country_sub = pd.read_csv('/kaggle/input/ken-jee-youtube-data/Aggregated_Metrics_By_Country_And_Subscriber_Status.csv', encoding='utf-8')
df_ts = pd.read_csv('/kaggle/input/ken-jee-youtube-data/Video_Performance_Over_Time.csv', encoding='utf-8')
df_comments = pd.read_csv('/kaggle/input/ken-jee-youtube-data/All_Comments_Final.csv', encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ken-jee-youtube-data/Aggregated_Metrics_By_Video.csv'

In [ ]:
#look at columns for each dataframe 
print(df_agg.columns)
print(df_agg_country_sub.columns)
print(df_ts.columns)
print(df_comments.columns)


In [ ]:
#the column headers have some extra non-ascii characters, we need to clean them up before we do our analysis
#this goes through each column and removes all the non-ascii characters 

newcols =[x.encode("ascii", "ignore").decode('utf-8') for x in df_agg.columns]
df_agg.columns = newcols

In [ ]:
df_agg.columns

In [ ]:
#Using .describe() gives us basic descriptive statistics for all of our numeric data. From this we can see which variables we might want to explore more. 
#We can see things like: which variables might have outliers, which might have skew, or which have a wide range of values
df_agg.describe()

## Single variable plots
After looking at the descriptive statistics, we may want to explore our numeric features more. To do this, we like to use single variable plots. These can help us understand the distributions of our data. If our data doesn't follow a normal distribution, we may want to make some transform it so it can be used by specific types of models like linear regresison. 

Looking at these charts can also help us to evaluate if there are outliers present in our data. 

First, we will look at histograms of our data. These help us to see skew as well as some outliers. I usually do this with every feature (if possible). Try more of these on your own! 

Next, we will explore some box plots to see if they tell us additional information

Finally, we will explore some distribution plots for categorical variables.

In [ ]:
#you can easily see the distribuiton of data in a column with panda's built in .hist() method. Here is the distribution for likes. There is clearly one sample that has way more like sthan the others, this is something we could look into more

df_agg.Likes.hist(bins = 100)

In [ ]:
#let's look at the same thing for average percentage viewed. This will have no outliers unlike the likes column. We are using matplotlib's hist function here instead of the integration through pandas. 
plt.hist(df_agg['Average percentage viewed (%)'])

In [ ]:
#this data has a significant amount of right skew, we may want to transform this data if we were planning to use linear regression.
df_agg['Impressions click-through rate (%)'].hist()

In [ ]:
#From this boxplot, we can see that there are quite a few outliers from our normal data. What does this tell us about the nature of likes on videos? Perhaps we shouldn't be trusting averages if something can skew so high. 
plt.boxplot(df_agg['Likes'])

In [ ]:
#this plot looks a lot more normal. We can se emost of the videos are viewed between 25-45% of the way through. We still have some outliers, what may be special or different about those? 
plt.boxplot(df_agg['Average percentage viewed (%)'])
#

In [ ]:
#with click through rate, we also see quite a few high outliers versus the median of / interquartile range.
plt.boxplot(df_agg['Impressions click-through rate (%)'])

In [ ]:
#To show how to do this with categorical data, let's make a categorical column.
#We can take our revenue data and make it into different categories that are relevant to me. Usually I'm interested if videos have made less than $100, 
#between $100-1000 and over $1000. 

#Let's quickly "engineer these categories for ourself"

#make bins from 0-100, 100-1000, and greater than 1000
bins = pd.IntervalIndex.from_tuples([(0, 100), (100, 1000), (1000,float("inf"))])
df_agg['rev_buckets'] = pd.cut(df_agg['Your estimated revenue (USD)'],bins)

#get count of number of videos by reveune bucket
rev_values = df_agg['rev_buckets'].value_counts()
rev_values.plot.bar()

#We can get a sense of the balance of categorical variables using a bar chart like this. 

## Relationships and Multi-Variable Plots
A big part of exploratory data analysis is seeing how mutliple variables are related. We can use multiple different types of plots to easily see these relationships. By understanding these relationships we can start to udnerstand which features may be releated or can serve to predict others. In my opinion, this is where buisiness value starts to emerge. 

In this part we will explore:

1) Scatter plots

2) Correlation Matrices 

3) Pivot Tables

4) Bar Charts

5) Line Charts
 

### Scatter Plots

We use these to see if there is a relationship between two datapoints. 

Let's take a look  at a few variables and see if they may be correlated. We will explore if the average percentage viewed of the video is related to the cost per milli on the video (the amount youtube makes for 1000 views)


In [ ]:
#create plot with matplotlib
plt.scatter(df_agg['Average percentage viewed (%)'] ,df_agg['CPM (USD)'])

In [ ]:
#let's do the same thing with seaborn so we can see a trendline. This will be easier for us to build multiple of these 
sns.regplot(x='Average percentage viewed (%)',y='CPM (USD)', data = df_agg)

#as we can see there is a very little correlation between the two variables 

In [ ]:
#Let's try another. How do variables RPM and CPM match up. RPM is how much the youtuber makes for an add. Again CPM is how much youtube sells the ad for. 
sns.regplot(x='RPM (USD)',y='CPM (USD)', data = df_agg)

# Here we can see a lot stronger relationship between the two rates. 
# I should probably look into that one value where YouTube is making a massive margin on my video. These are the types of insights you can find in these graphs


### Correlation Matrices 

Scatter plots are great for comparing two variables. Often we have many different variables that we want to see relationships between. 


In this case we would use a correlation matrix. Let's do a correlation matrix for all of our value in the dataset. 


In [ ]:
#first, we get the correlations between our datapoints. We can see all the relationships that will be polotted via the pearson correlation coefficient 
corr = df_agg.corr()

sns.heatmap(corr)

#this looks pretty awful, let's improve it's look

In [ ]:
#A better example (formatting used in below chart) - https://seaborn.pydata.org/examples/many_pairwise_correlations.html 

sns.set_theme(style="white")

# Compute the correlation matrix
corr = df_agg.corr()

# Generate a mask for the upper triangle (otherwise this looks like the square we had above and is redundant)
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure 
f, ax = plt.subplots(figsize=(15, 10))

# Generate a custom diverging colormap (choose colors here)
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio 
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, annot_kws={"fontsize":8})

#obviously many of thes variables are HIGHLY correlated. Something we may want to explore is why Average percentage viewed is negatively related to RPM

###Pivot Tables
Sometimes we want to cut the data and compare how variables stack up. For example we may want to see which countries have the highest view duration. We can do this easily with pivot tables. 

We will use the df_agg_country_sub dataset to explore this.


In [ ]:
#pivot table to explore values. A basic pivot table takes the average of these different categories we choose.

pd.pivot_table(df_agg_country_sub, index = 'Country Code', values = 'Average View Percentage')

In [ ]:
#this time, let's compare average view percentage by country code and subscriber status.
pd.pivot_table(df_agg_country_sub, index = 'Country Code', columns = 'Is Subscribed',values = 'Average View Percentage')

#we could plot this to see which countries have the biggest difference in subscribed watch time for subscribed vs not subscribed viewers 

In [ ]:
#Let's plot a slightly simpler graph. Let's just look at if subscribers or non-subsribers watch my videos for longer
pd.pivot_table(df_agg_country_sub, index = 'Is Subscribed', values = 'Average View Percentage').plot.bar()

#it appears that subscribers to my channel watch my videos for longer than non-subscribers. This is another way that we can use a bar chart to compare multiple features

### Line Charts
Often, we want to see how variables change over time. Line charts are great for this. Time is an important feature in many models! 

We will use the df_ts dataset for this analys.

In [ ]:
#First we need to make sure our date field is in the date time format. We do this by converting our string to datetime
df_ts['Date'] = pd.to_datetime(df_ts['Date'])

#let's look at user subscriptions removed and see if there is any trend there. We will also compare this with user likes removed to see if we can find anything interesting

#first, we have to aggregate these by video. We do this with a pivot table.
rm_x_date = pd.pivot_table(df_ts, index='Date',values = 'User Subscriptions Removed', aggfunc ='sum').reset_index()

#next we visualize this data with seaborn 
sns.lineplot(data=rm_x_date,x='Date', y='User Subscriptions Removed')

In [ ]:
#now let's compare this with two other related metrics, video likes removed and dislikes

likes_rm_date = pd.pivot_table(df_ts, index='Date',values='Video Likes Removed', aggfunc = 'sum').reset_index()
dislikes_date = pd.pivot_table(df_ts, index='Date',values='Video Dislikes Added', aggfunc = 'sum').reset_index()

sns.lineplot(data=rm_x_date,x='Date', y='User Subscriptions Removed')
sns.lineplot(data=likes_rm_date,x='Date', y='Video Likes Removed')
sns.lineplot(data=dislikes_date,x='Date', y='Video Dislikes Added')


In [ ]:
#This was a bit messy, let's compare it over months instead of days.
df_ts['Month_Year'] = df_ts['Date'].dt.to_period('M')

#create new pivot tables 
rm_x_date = pd.pivot_table(df_ts, index='Month_Year',values = 'User Subscriptions Removed', aggfunc ='sum').reset_index()
likes_rm_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Likes Removed', aggfunc = 'sum').reset_index()
dislikes_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Dislikes Added', aggfunc = 'sum').reset_index()

#create 3 separate line plots with pandas built in visualization tool 
rm_x_date.plot(x='Month_Year',y='User Subscriptions Removed')
likes_rm_date.plot(x='Month_Year',y='Video Likes Removed')
dislikes_date.plot(x='Month_Year',y='Video Dislikes Added')


In [ ]:
#This was a bit messy, let's compare it over months instead of days. To get everything on one graph, we need to make it so every video has the same day.
#Just different months and years 
df_ts['Month_Year'] = df_ts['Date'].apply(lambda x: x.replace(day=1))

rm_x_date = pd.pivot_table(df_ts, index='Month_Year',values = 'User Subscriptions Removed', aggfunc ='sum').reset_index()
likes_rm_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Likes Removed', aggfunc = 'sum').reset_index()
dislikes_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Dislikes Added', aggfunc = 'sum').reset_index()

#create 3 line plots with seaborn all on one graph 
sns.lineplot(data=rm_x_date,x='Month_Year', y='User Subscriptions Removed', label ='Subs Removed')
sns.lineplot(data=likes_rm_date,x='Month_Year', y='Video Likes Removed', label = 'Likes Removed')
sns.lineplot(data=dislikes_date,x='Month_Year', y='Video Dislikes Added', label = 'Dislikes')

#it seems like there is a big spike in unsubscribes and negative comments during this period. We should explore this more.
#maybe we should divide these all by views to determine if the spike is only related to a spike in viewership as well.
#or maybe a single video caused a lot of negative things this period 

In [ ]:
#Let's normalize this by views 

df_ts['Month_Year'] = df_ts['Date'].apply(lambda x: x.replace(day=1))

monthly_views = pd.pivot_table(df_ts, index = 'Month_Year', values = 'Views', aggfunc = 'sum')
rm_x_date = pd.pivot_table(df_ts, index='Month_Year',values = 'User Subscriptions Removed', aggfunc ='sum') / monthly_views.values
likes_rm_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Likes Removed', aggfunc = 'sum') / monthly_views.values
dislikes_date = pd.pivot_table(df_ts, index='Month_Year',values='Video Dislikes Added', aggfunc = 'sum') / monthly_views.values

#create 3 line plots with seaborn all on one graph 
sns.lineplot(data=rm_x_date,x='Month_Year', y='User Subscriptions Removed', label ='Subs Removed')
sns.lineplot(data=likes_rm_date,x='Month_Year', y='Video Likes Removed', label = 'Likes Removed')
sns.lineplot(data=dislikes_date,x='Month_Year', y='Video Dislikes Added', label = 'Dislikes')

#we can see that there was still a spike in the normalized metrics during the same point in the year. It is worth exploring this further! 

## Summary
In this segment you learned the following:
- How to analyze individual features using histograms, boxplots, and bar charts
- How to see relationships between features using scatterplots, correlation heatmaps, bar charts, and line plots 
- How to use these charts to seek additional insight from your data and explore what relationships to dive deeper into 

## Comment Data

## Your Turn! 
### I started you off with the basics of Exploratory Data Analsys (EDA), it's your turn to add your insights to the analysis. Excited to see what you come up with!!
I'm constantly thinking about questions like:
- What topics get the most viewership?
- What do are people asking for in the comments?
- What impacts watch time and click through rate?
- Can we predict if a title will be clickable? 
- What thumbnails are most appealing (thumbnail data available in df_agg)
- What is different about my "viral" videos and normal videos
- Who is the core audience of my channel?
